# Data Commons RESTful

Data Commons SPARQL is done via POST to an endpoint


```bash
curl -X POST 'https://api.datacommons.org/query' \
-d '{"sparql": "SELECT ?name \
                WHERE { \
                  ?state typeOf State . \
                  ?state dcid geoId/06 . \
                  ?state name ?name \
                }"}'
```


## References

* [DC REST](https://docs.datacommons.org/api/rest/)
* [DC SPARQL](https://docs.datacommons.org/api/rest/query.html)
* [External links](https://docs.datacommons.org/bigquery/query_join_your_data.html)

## Big Query UI

[Analytics Hub](https://console.cloud.google.com/bigquery/analytics-hub/exchanges(analyticshub:projects/841968438789/locations/us/dataExchanges/data_commons_17d0b72b0b2/listings/data_commons_1803e67fbc9))


In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import requests
import numpy as np
import json
import geopandas
import matplotlib.pyplot as plt
import shapely

dbsparql = "http://dbpedia.org/sparql"
geodex = "http://graph.geodex.org/blazegraph/namespace/nabu/sparql"
agu = "http://abstractsearch.agu.org:8890/sparql"
datacommons = "https://api.datacommons.org/query"

In [2]:
#@title
def get_sparql_dataframe(service, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

## SPARQL call example

This is neither standard SPARQL, no lack of namespace, and simply the fact 
this must be sent in as a JSON structure to a service.   Information on the query endpoint can be found at https://docs.datacommons.org/api/rest/v1/query

Note:  This means you can not use as part of a SPARQL federated query.  Since you will need to 
do the integration in code then, you may wish to use some of the other API options such as the 
pandas integration APIs.  

In [3]:
 

sq = """SELECT ?name
                WHERE {
                  ?state typeOf State .
                  ?state dcid geoId/06 .
                  ?state name ?name
                }
"""
 

jsq = {"sparql": sq}

r = requests.post(datacommons, json=jsq)
r.status_code
r.json()

{'header': ['?name'], 'rows': [{'cells': [{'value': 'California'}]}]}

## REST calls

### GET

In [4]:
rurl = "https://api.datacommons.org/stat/series?place=geoId/39&stat_var=RetailDrugDistribution_DrugDistribution_Naloxone&unit=Grams"

r = requests.get(rurl)
r.status_code
r.json()

{'series': {'2007-01': 705.03,
  '2006-10': 568.93,
  '2007-07': 1028.5,
  '2006-01': 389.59,
  '2006-04': 415.06,
  '2007-04': 870.28,
  '2006-07': 482.37,
  '2007-10': 1293.8}}

### POST

see: https://docs.datacommons.org/api/rest/stat_all.html

curl example:

```curl -X POST 'https://api.datacommons.org/stat/all' -d '{ "places": ["geoId/05", "geoId/06085"], "stat_vars": ["Count_Person_Male", "Count_Person_Female"]}'```

In [5]:
rj = { "places": ["geoId/39003", "geoId/39005","geoId/39013","geoId/39015","geoId/39017"], "stat_vars": ["Count_Person", "Median_Age_Person"]}

r = requests.post('https://api.datacommons.org/stat/all', json=rj)
r.status_code
r.json()

{'placeData': {'geoId/39003': {'statVarData': {'Median_Age_Person': {'sourceSeries': [{'val': {'2015': 38.1,
        '2020': 39.4,
        '2021': 39.5,
        '2016': 38.2,
        '2012': 38.1,
        '2022': 39.8,
        '2019': 39.2,
        '2017': 38.6,
        '2014': 38.2,
        '2013': 38.1,
        '2011': 38.1,
        '2018': 39},
       'measurementMethod': 'CensusACS5yrSurvey',
       'importName': 'CensusACS5YearSurvey',
       'provenanceDomain': 'census.gov',
       'unit': 'Year',
       'provenanceUrl': 'https://www.census.gov/programs-surveys/acs/data/data-via-ftp.html'},
      {'val': {'2020': 39.4,
        '2014': 38.2,
        '2012': 38.1,
        '2011': 38.1,
        '2017': 38.6,
        '2013': 38.1,
        '2010': 38.1,
        '2016': 38.2,
        '2021': 39.5,
        '2019': 39.2,
        '2018': 39,
        '2015': 38.1},
       'measurementMethod': 'CensusACS5yrSurveySubjectTable',
       'importName': 'CensusACS5YearSurvey_SubjectTables_S0101',